# Sobrecarga de operadores

O Python permite definir o que deve ser executado quando um operador for usado em conjunto com os objetos de uma classe.

## 1. Operadores aritméticos

Por exemplo, ao encontrar o código `a + b` o Python tentará executar um código da seguinte forma `a.__add__(b)`. Portanto, se definirmos o método `__add__` para a classe do objeto `a`, podemos definir como esses tipos de objetos fazem somas.

A classe abaixo define uma operação de soma que não é matematicamente correta.

In [2]:
class StonedNumber:
    def __init__(self, val):
        self._val = val
    def get_value(self):
        return self._val
    def set_value(self, val):
        self._val = val
    def __add__(self, other):
        return self._val + other._val / 2

Com essa definição, podemos executar o método pela forma tradicional:

In [3]:
a = StonedNumber(2); b = StonedNumber(8)

In [4]:
a.get_value(), b.get_value()

(2, 8)

In [ ]:
a.__add__(b)

Ou podemos simplesmente usar o operador `+`:

In [ ]:
a + b

### 1.1. Variantes

No caso de operadores aritméticos, podemos definir para cada um **três** operações. Para o exemplo do `+`, temos as seguintes possibilidades (sendo `obj` um objeto de classe e `x` o objeto que queremos somar com `obj`):

    obj + x
    x + obj
    obj += x

Podemos definir as operações separadamente (se necessário) para cada um desses casos, com os métodos `__add__`, `__radd__` e `__iadd__`, respectivamente.

In [ ]:
class Useless:
    def __init__(self, ini):
        self._val = ini
    def value(self):
        return self._val
    def set_value(self, newv):
        self._val = newv
    def __add__(self, other):
        print('Useless.__add__')
        if isinstance(other, Useless):
            return Useless(self._val + other._val)
        else:
            return Useless(self._val + other)
    def __radd__(self, other):
        print('Useless.__radd__')
        return self.__add__(other)
    
    def __iadd__(self, other):
        print('Useless.__iadd__')
        if isinstance(other, Useless):
            self._val += other._val
        else:
            self._val += other
        return self
    def __sub__(self, other):
        print('Useless.__sub__')
        if isinstance(other, Useless):
            return Useless(self._val - other._val)
        else:
            return Useless(self._val - other)
    def __rsub__(self, other):
        print('Useless.__rsub__')
        return Useless(other - self._val)
    def __isub__(self, other):
        print('Useless.__isub__')
        if isinstance(other, Useless):
            self._val -= other._val
        else:
            self._val -= other
        return self

Note como os métodos do tipo `a + b` retornam um novo objeto, enquanto os método do tipo `+=` retornam `self`. Isso é a forma correta e deve ser seguida.

Veja as mensagens nas execuções abaixo, para entender qual método é chamado em qual situação (veja o código acima).

In [ ]:
a = Useless(5); b = Useless(3)

In [ ]:
c = a + b

In [ ]:
c.value()

In [ ]:
d = a + 3

In [ ]:
d.value()

In [ ]:
e = 3 + a

In [ ]:
e.value()

In [ ]:
c = a - b

In [ ]:
c.value()

In [ ]:
d = a - 2

In [ ]:
d.value()

In [ ]:
e = 2 - b

In [ ]:
e.value()

In [ ]:
a += 4

In [ ]:
a.value()

In [ ]:
a -= b
a.value()

Uma das características do `__iadd__` que o diferencia do `__add__` é que ele em geral não cria um novo objeto (a exceção é quando o objeto é de um tipo imutável, como string ou int, onde um novo objeto deve ser criado).

In [ ]:
b = a

In [ ]:
b is a

In [ ]:
a += 1

In [ ]:
b is a

In [ ]:
b = b + a

In [ ]:
b is a

## 2. Diversos operadores

Abaixo uma tabela resumida das operações aritméticas:

| Operação | Tradução |
|----------|----------|
| `a + b`  | `a.__add__(b)` |
| `a - b`  | `a.__sub__(b)` |
| `a * b`  | `a.__mul__(b)` |
| `a / b`  | `a.__truediv__(b)` |
| `a // b`  | `a.__floordiv__(b)` |
| `a % b`  | `a.__mod__(b)` |
| `-a`  | `a.__neg__()` |
| `+a`  | `a.__pos__()` |
| `a ** b`  | `a.__pow__(b)` |

Também operações lógicas bit a bit podem ser sobrecarregadas. Por exemplo:

| Operação | Tradução |
|----------|----------|
| `a & b`  | `a.__and__(b)` |
| `a \| b`  | `a.__or__(b)` |
| `a ^ b`  | `a.__xor__(b)` |
| `a << b`  | `a.__lshift__(b)` |
| `a >> b`  | `a.__rshift__(b)` |
| `~a` | `a.__inv__()` |

Quando apropriado, temos também as versões `__r*` e `__i*`  desses métodos.

## 3. Indexação

### 3.1. Leitura

Um outro operador que pode ser definido para suas classes é o de indexação, chamado quando usamos o objeto da forma `obj[i]` (para leitura). O método a ser implementado se chama `__getitem__` e deve receber o índice do ítem a ser retornado.

O caso mais simples é quando queremos apenas indexar com um valor, como no caso da classe abaixo, que representa uma lista com os quadrados dos valores de `0` a `maximo - 1`.

In [ ]:
class Squares:
    def __init__(self, max_):
        self._max = max_
    def __getitem__(self, i):
        if i < 0:
            i += self._max
        if i < 0 or i >= self._max: 
            raise IndexError()
        return i ** 2

In [ ]:
q = Squares(10)
for i in range(10): print(q[i], end=' ')

In [ ]:
q[1], q[0], q[9]

In [ ]:
q[10]

Por ter o método `__getitem__`, os objetos do tipo `Squares` podem ser usados como uma lista em operações `for`.

In [ ]:
for x in q:
    print(x)

Este foi apenas um exemplo. A forma mais simples de realizar isso é:

In [ ]:
for x in (i ** 2 for i in range(10)):
    print(x)

### 3.2. Escrita

Se quisermos **alterar** o valor de um índice, devemos usar o `__setitem__`.

Esse método não faz sentido para o objeto de quadrados, então não o definimos, o que faz com que atribuições sejam proibidas.

In [ ]:
q[2] = 5

Vejamos um exemplo que possibilita alteração, na classe abaixo que guarda o valor absoluto dos elementos inseridos.

In [ ]:
class AbsoluteValues:
    def __init__(self, size):
        self._values = [0 for i in range(size)]
    def __getitem__(self, i):
        return self._values[i]
    def __setitem__(self, i, val):
        self._values[i] = abs(val)

In [ ]:
va = AbsoluteValues(5)

In [ ]:
va[0:5]

In [ ]:
va[3] = -4; va[1] = 10

In [ ]:
va[0:5]

O código anterior nos lembra que as listas de Python aceitam indexação com faixas de índices (denominados *slices*).

Para permitir indexação com *slices* na sua classe, o seu método `__getitem__` deve saber lidar com um objeto do tipo `slice` sendo passado como índice ao invés de um inteiro.

No código anterior isso funciona pois o `__getitem__` usa o valor recebido no argumento `i` apenas para diretamente indexar uma lista, que já sabe lidar com *slices*.

In [ ]:
va[-3]

In [ ]:
va[1:-2]

No caso normal, precisamos programar especialmente o `__getitem__` para funcionar corretamente com *slices*.

Os objetos do tipo `slice` possuem atributos `start`, `stop` e `step` que definem os valores do slice (similares aos parâmetros de `range`). Os valores desses atributos serão `None` se não forem especificados (por exemplo, usando `[1:10:2]` teremos `start=1`, `stop=10` e `step=2`; com `[:10]` teremos `start=None`, `stop=10` e `step=None`.

No exemplo abaixo, redefinimos `Squares` para aceitar slices.

In [ ]:
class Squares:
    def __init__(self, max_):
        self._max = max_
    def __getitem__(self, i):
        if isinstance(i, slice):
            # We got a slice
            # Take care of the None values
            start = 0 if i.start is None else i.start
            stop = self._max if i.stop is None else i.stop
            step = 1 if i.step is None else i.step
            # The negative values must be converted
            if start < 0: start += self._max
            if stop < 0: stop += self._max
            if (start < 0 or start > self._max or
                stop < 0 or stop > self._max):
                raise IndexError()
            # Returns a generator with the required values
            return (x ** 2 for x in range(start, stop, step))
        else:
            if i < 0:
                i += self._max
            if i < 0 or i >= self._max: 
                raise IndexError()
            return i ** 2

In [ ]:
q = Squares(10)

In [ ]:
m = list(i**2 for i in range(10))
m

In [ ]:
[q[i] for i in range(10)]

In [ ]:
list(q[1:3])

In [ ]:
list(q[:-1])

In [ ]:
list(q[::2])

In [ ]:
q[2]

In [ ]:
list(q[-5:-1])

In [ ]:
list(q[-5:1:-1])

In [ ]:
list(q[:])

In [ ]:
q[:15]

# Exercício

1. O código abaixo usa funções (métodos) com nomes arbitrários para as operações aritméticas. Re-escreva o código para substituir esses métodos por operadores aritméticos. (Não ligue para o fato de que o código é inútil.)
```python
class GrowingInt:
    def __init__(self, val):
        self._value = val

    def get_value(self):
        return self._value

    def add(self, other):
        if other._value > 0:
            result = GrowingInt(self._value + other._value)
        else:
            result = GrowingInt(self._value)
        return result

    def sub(self, other):
        if other._value < 0:
            result = GrowingInt(self._value - other._value)
        else:
            result = GrowingInt(self._value)
        return result

    def mul(self, other):
        if ((self._value > 0 and other._value > 1)
            or (self._value < 0 and other._value < 1)):
            result = GrowingInt(self._value * other._value)
        else:
            result = GrowingInt(self._value)
        return result

    def div(self, other):
        if ((self._value > 0 and other._value < 1)
            or (self._value < 0 and other._value > 1)):
            result = GrowingInt(self._value * other._value)
        else:
            result = GrowingInt(self._value)
        return result

a = GrowingInt(10)
b = GrowingInt(2)
c = GrowingInt(-3)
d = GrowingInt(0.5)

apb = a.add(b); print(apb.get_value())
apc = a.add(c); print(apc.get_value())
cpb = c.add(b); print(cpb.get_value())
cpc = c.add(c); print(cpc.get_value())
amb = a.mul(b); print(amb.get_value())
amc = a.mul(c); print(amc.get_value())
amd = a.mul(d); print(amd.get_value())
cmb = c.mul(b); print(cmb.get_value())
cmd = c.mul(d); print(cmd.get_value())
```

2. Escreva uma classe `Multiples` cujos objetos quando criados com `Multiples(a, n)` representam a sequência dos `n` primeiros múltiplos de `a` começando de 0. Isto é, se `m3 = Multiples(3, 10)`, então `m3[0]` deve ser 0, `m3[1]` dever ser 3, etc, até `m3[9]` que deve ser 27. Fora dessa faixa (incluindo negativos) os índices devem ser inválidos. O objeto deve também funcionar com *slice*, com por exemplo `m3[1:5]` devendo retornar um gerador que fornece os valores múltiplos de 3 de 3 (incluido) a 15 (excluido).